# PHysical Activity Monitoring (PAM)

This notebook will serve as a reference on how to use phyisical activity monitoring algortihm.

The example illustrates how the physical activity monitoring (PAM) algorithm is used to determine the intensity level of sedentary, light, moderate, and vigorous physical activities using body acceleration recorded with a triaxial accelerometer worn on the wrist. The physical activity monitoring algorithm is implemented in the main module `NGMT\ngmt\modules\PAM.py`.

The algorithm determines the intensity level of the physical activities based on the following steps. It starts by loading the input_data, which includes data with a time index along with accelerometer data (N, 3) for x, y, and z axes. The other input of the algorithm is sampling_frequency, which is in Hz, and the default value is 100. Another input of the algorithm is thresholds, which should be provided as a dictionary containing threshold values for physical activity detection. Epoch length, which is defined in seconds, is the next input of the algorithm, as shown with epoch_duration. The default value is 5 seconds. The last input of the algorithm is plot_results, which, if set to True, generates a plot showing the average Euclidean Norm Minus One (ENMO) per hour for each date. The default is True.

The following steps are taken to preprocess the input signal. First, the sample-level Euclidean norm (EN) of the acceleration signal across the x, y, and z axes is calculated. Next, a fourth-order Butterworth low-pass filter with a cut-off frequency of 20Hz is applied to remove noise. This filter is applied to the vector magnitude scores, rather than the individual axes. To separate out the activity-related component of the acceleration signal, the ENMO index is calculated. The Euclidean Norm Minus One (ENMO) is a summary metric for acceleration data and represents the vector magnitude of 3 axial measures minus the contribution of gravity (1 g). Then, negative values of the ENMO are truncated to zero. Finally, the calculated indices are multiplied by 1000 to convert the units of the acceleration from g to milli-g.

The algorithm continues by classifying the intensity of the physical activities based on the calculated ENMO values. Using the activity_classification function, the ENMO time-series data is then expressed in 5-second epochs. Epochs with a length of 5 seconds are used for summarizing the data, as this epoch length has been suggested to be able to capture shorter bouts of activities. The greater the intensity of movement and duration of activity in the summed 5-second epochs are, the greater the ENMO value is. Then, the intensity of activities as the time distribution of ENMO using 5-second epochs is used to classify activities based on different thresholds. In the analysis of intensity distribution, the following thresholds are used for categorization: sedentary activity < 45 milli-g, light activity 45–100 milli-g, moderate activity 100–400 milli-g, vigorous activity > 400 milli-g.

Finally, the algorithm takes the last steps to classify different levels of activities along with the time spent on each activity level for each day. The algorithm also visualizes the averaged ENMO values for each day.


## Import libraries
The necessary libraries such as pandas, os and physical activity monitoring (PAM) are imported. Make sure that you have all the required libraries and modules installed before running this code. You also may need to install the 'ngmt' library and its dependencies if you haven't already.

In [1]:
import pandas as pd
import numpy as np
from ngmt.modules import PAM
import os
from ngmt.utils.fairpark import load_fairpark_data

## Load Data
The data, including time index and accelerometer data (N, 3) for the x, y, and z axes, is loaded as a numpy.ndarray.

In [ ]:
# The data folder path
data_folder_path = r"C:\Users\Project\Desktop\sub_001"

# Function to load motion data
acc_data, sampling_frequency = load_fairpark_data(data_folder_path)

## Apply the physical activity monitoring algorithm
Now, we are running physical activity monitoring (PAM) algorithm from main module (`NGMT.ngmt.modules.PAM.Physical_Activity_Monitoring`). The inputs of the algorihm are as follows. The input_data, includes data with a time index along with accelerometer data (N, 3) for x, y, and z axes. The sampling_frequency is the sampling frequecy of the acceleration data which is defined in Hz, and the default value of it is 100 Hz. The next input, thresholds, are provided as a dictionary containing threshold values for physical activity detection. The epoch_duration is the epoch length in seconds and the default value is 5 seconds. The last input of the algorithm is plot_results, which, if set to True, generates a plot showing the average Euclidean Norm Minus One (ENMO) per hour for each day. The default is True.




In [ ]:
# Use Physical_Activity_Monitoring to classify physical activity levels
phyam_results = PAM.Physical_Activity_Monitoring(
    input_data=acc_data,
    sampling_frequency=100,
    thresholds={
        "sedentary_threshold": 45,
        "light_threshold": 100,
        "moderate_threshold": 400,
    },
    epoch_duration=5,
    plot_results=True,
)

In [ ]:
# Display average ENMO values for each activity level for each day.
for index, row in phyam_results.iterrows():
    print(f"Date: {row['date']}")

    # Check if there are NaN values for mean acceleration
    if not np.isnan(row["sedentary_mean_acc"]):
        print(f"Mean ENMO for Sedentary: {row['sedentary_mean_acc']} mili-g")
        print(f"Time Spent for Sedentary: {row['sedentary_spent_time_minute']} minutes")

    if not np.isnan(row["light_mean_acc"]):
        print(f"Mean ENMO for Light Activity: {row['light_mean_acc']} mili-g")
        print(
            f"Time Spent for Light Activity: {row['light_spent_time_minute']} minutes"
        )

    if not np.isnan(row["moderate_mean_acc"]):
        print(f"Mean ENMO for Moderate Activity: {row['moderate_mean_acc']} mili-g")
        print(
            f"Time Spent for Moderate Activity: {row['moderate_spent_time_minute']} minutes"
        )

    if not np.isnan(row["vigorous_mean_acc"]):
        print(f"Mean ENMO for Vigorous Activity: {row['vigorous_mean_acc']} mili-g")
        print(
            f"Time Spent for Vigorous Activity: {row['vigorous_spent_time_minute']} minutes"
        )

    print("=" * 40)